# Imports

In [ ]:
import numpy as np
import pandas as pd


from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Load dataset and split into train and test

In [ ]:
iris = load_iris()
X = iris['data']
y = iris['target']
names = iris['target_names']
feature_names = iris['feature_names']

# Scale data to have mean 0 and variance 1
# which is importance for convergence of the neural network
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data set into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

print(X_train.shape, X_test.shape)

## Create an MLFoundry run and log dataset

In [ ]:
!pip install --quiet "mlfoundry>=0.3.33,<0.4.0"

In [ ]:
import mlfoundry as mlf

mlf.login()
mlf_client = mlf.get_client()

ML_FLOW = "gh-demo-test"
run = mlf_client.create_run(ML_FLOW, "base-run")

In [ ]:
run.log_dataset("train", X_train, y_train)
run.log_dataset("test", X_test, y_test)

## Create the model

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

class Model(nn.Module):
    def __init__(self, input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim, 50)
        self.layer2 = nn.Linear(50, 30)
        self.layer3 = nn.Linear(30, 3)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1)
        return x

LEARNING_RATE = 0.001
EPOCHS  = 100

model     = Model(X_train.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_fn   = nn.CrossEntropyLoss()
print(model)

## Log the hyper-parameters using MLFoundry

In [ ]:
run.log_params({
    'learning_rate': LEARNING_RATE,
    'epochs': EPOCHS
})

In [ ]:
run.set_tags({
    'feature_names': feature_names,
    'target_names': names
})

## Write a function to log confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

def log_confusion_matrix(y_true, y_pred, run=None, step=0):
  fig, ax = plt.subplots(figsize=(5, 5))
  ax.matshow(confusion_matrix(y_true, y_pred), cmap=plt.cm.Reds, alpha=0.5)
  
  if (run): run.log_plots({'confusion_matrix': fig}, step)

  plt.close()

## Train the model
After every epoch:
  * Log metrics using `log_metrics`
  * Log test dataset confusion matrix using `log_plots`

In [ ]:
X_train_tensor = Variable(torch.from_numpy(X_train)).float()
y_train_tensor = Variable(torch.from_numpy(y_train)).long()
X_test_tensor  = Variable(torch.from_numpy(X_test)).float()
y_test_tensor  = Variable(torch.from_numpy(y_test)).long()

In [ ]:
%matplotlib inline 
import tqdm

loss_list     = np.zeros((EPOCHS,))
accuracy_list = np.zeros((EPOCHS,))

for epoch in tqdm.trange(EPOCHS):
    y_pred = model(X_train_tensor)
    loss = loss_fn(y_pred, y_train_tensor)

    # Zero gradients
    optimizer.zero_grad() 
    loss.backward()
    optimizer.step()

    with torch.no_grad():
        y_train_pred = model(X_train_tensor)
        y_test_pred = model(X_test_tensor)
        
        
        train_correct = (torch.argmax(y_train_pred, dim=1) == y_train_tensor).type(torch.FloatTensor) 
        test_correct = (torch.argmax(y_test_pred, dim=1) == y_test_tensor).type(torch.FloatTensor)

        # log metrics and plots every 10 epochs
        if (epoch % 10 == 0):
          log_confusion_matrix(y_test_tensor, torch.argmax(y_test_pred, dim=1), run, epoch)
          run.log_metrics({
              'loss': loss.item(),
              'train/accuracy': train_correct.mean().item(),
              'test/accuracy': test_correct.mean().item()
          }, epoch)

## Searching for optimal hyper parameters

In [ ]:
LEARNING_RATES = [0.001, 0.01, 0.1]
EPOCHS_VALUES = [10, 100]

for i, LR in enumerate(LEARNING_RATES):
  for j, EPOCHS in enumerate(EPOCHS_VALUES):
    run = mlf_client.create_run(ML_FLOW, f"param-search-{i}-{j}")

    run.set_tags({
        'type': 'param-search'
    })

    run.log_params({
    'learning_rate': LR,
    'epochs': EPOCHS
    })

    model     = Model(X_train.shape[1])
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    loss_fn   = nn.CrossEntropyLoss()

    for epoch in range(EPOCHS):
      y_pred = model(X_train_tensor)
      loss = loss_fn(y_pred, y_train_tensor)

      # Zero gradients
      optimizer.zero_grad() 
      loss.backward()
      optimizer.step()
    
    y_test_pred = model(X_test_tensor)
    test_correct = (torch.argmax(y_test_pred, dim=1) == y_test_tensor).type(torch.FloatTensor)

    run.log_metrics({
        'accuracy': test_correct.mean().item()
    })

In [ ]:
run.log_model(model, 'pytorch')

## Deploy a demo app

In [ ]:
!pip install --quiet gradio==3.0.13
!pip install --quiet servicefoundry==0.1.65

In [ ]:
import servicefoundry.core as sfy
sfy.login()

In [ ]:
%%writefile webapp.py
import gradio as gr
import pandas as pd
import mlfoundry as mlf
import torch
import json

CLASS_NAMES = ['setosa', 'versicolor', 'virginica']

TFY_API_KEY = '<use-your-api-key>'
RUN_ID = '<run_id-of-relevant-run>'

client = mlf.get_client(api_key=TFY_API_KEY)
run = client.get_run(RUN_ID)
model = run.get_model()

def predict_species(f1, f2, f3, f4):
    y_pred = model(torch.Tensor([[f1, f2, f3, f4]]))
    return CLASS_NAMES[torch.argmax(y_pred, dim=1)[0].item()]


dataset = run.get_dataset('train')
examples = dataset.features.sample(5).values.tolist()
app = gr.Interface(fn=predict_species, title="Iris Classification", inputs=[gr.Number(label="sepal length (cm)"), gr.Number(label="sepal width (cm)"), gr.Number(label="petal length (cm)"), gr.Number(label="petal width (cm)")], outputs=[gr.Textbox(label="Answer")], examples=examples)


### Create a Servicefoundry workspace

A Servicefoundry workspace is a collection of related services that share the same set of permissions.

To create a workspace:

1. Go to <a href="https://app.truefoundry.com/workspace">ServiceFoundry dashboard</a>

2. Click on `Create Workspace` to create a new workspace.

3. Once the workspace is created, copy the FQN of the workspace. We shall use this to deploy our webapp and service to the workspace.

In [ ]:
WEBAPP_NAME = "gradio-app"
WORKSPACE_FQN = input("Input workspace FQN copied from the dashboard ")

In [ ]:
requirements = sfy.gather_requirements("webapp.py")
webapp = sfy.Gradio("webapp.py", requirements, sfy.Parameters(
    name=WEBAPP_NAME,
    workspace=WORKSPACE_FQN,
    cpu=sfy.CPU(required=1.0),
    memory=sfy.Memory(required=1024 * 1000 * 1000)
))

webapp.deploy()

In [ ]:
# helper fn to generate service url
def get_service_url(workspace_fqn, service_name):
  _, cluster, ws_name = workspace_fqn.split(':')
  return f'https://{service_name}-{ws_name}.{cluster}.production.truefoundry.cloud'

In [ ]:
print(f'Once deployed, you can access the webapp at {get_service_url(WORKSPACE_FQN, WEBAPP_NAME)}')